*DATA INGESTION*

In [1]:
import pandas as pd
!pip install -U pandasql 
import pandasql as ps
from pandasql import sqldf
from google.colab import files


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26787 sha256=88685072f6ea2cd065d07db600ead60d993b33ada3cca389527d8c7f6eee4c04
  Stored in directory: /root/.cache/pip/wheels/ed/8f/46/a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql


In [2]:


#file paths

meta_browser = "/content/drive/MyDrive/Colab Notebooks/dataviz_challange/meta_browser.csv"
meta_device = "/content/drive/MyDrive/Colab Notebooks/dataviz_challange/meta_device.csv"
meta_os = "/content/drive/MyDrive/Colab Notebooks/dataviz_challange/meta_os.csv"
dataset = "/content/drive/MyDrive/Colab Notebooks/dataviz_challange/dataset.csv"

#read files

meta_browser = pd.read_csv(meta_browser)
meta_device = pd.read_csv(meta_device)
meta_os = pd.read_csv(meta_os)
dataset_csv=pd.read_csv(dataset, parse_dates=['giorno_min',"giorno_max"])

print(meta_browser.head())
print(meta_device.head())
print(meta_os.head())
print(dataset_csv.head())

#dataset_csv will be the fact table of our schema, while meta_browser, meta_device and meta_os will be the dimension

#get info regarding datatypes
print(dataset_csv.info())
print(meta_browser.info())
print(meta_device.info())
print(meta_os.info())


#split prima_citta in two different columns: prima_citta_stato_cod and prima_citta_cod

dataset_csv[['prima_citta_stato_cod',"prima_citta_cod"]]=dataset_csv["prima_citta"].str.split("-", expand=True)
dataset_csv.drop('prima_citta', inplace=True, axis=1)
dataset_csv.head()

# rename ?uuid into user_id
dataset_csv = dataset_csv.rename(columns={'?uuid': 'user_id'})

dataset_csv.head(9)


   id        name
0   1  Explorer 3
1   2  Explorer 4
2   3    Explorer
3   4  Explorer 5
4   6  Explorer 6
   id                name
0   0        Not detected
1   1              Mobile
2   2  Desktop and Laptop
3   3             Unknown
4   4                  TV
   id          name
0   1    Windows XP
1   2  Windows 2000
2   3    Windows 98
3   4    Windows ME
4   5    Windows 95
   ?uuid giorno_min giorno_max  durata_vita  conteggio_giorni_totale  \
0      1 2019-05-27 2019-05-27            1                        1   
1      2 2019-05-31 2019-06-13           14                        3   
2      3 2019-04-26 2019-10-06           46                        5   
3      4 2019-05-19 2019-05-19            1                        1   
4      5 2019-05-06 2019-10-06            6                        2   

   device_type  browser_type  os_type  site_giallozafferano  site_repubblica  \
0          5.0         128.0    132.0                     0                0   
1          2.0         

,user_id,giorno_min,giorno_max,durata_vita,conteggio_giorni_totale,device_type,browser_type,os_type,site_giallozafferano,site_repubblica,...,site_gazzetta,site_sole24ore,site_pourfemme,site_alfemminile,site_corrieresport,site_cookaround,site_autoscout,site_staseratv,prima_citta_stato_cod,prima_citta_cod
0,1,2019-05-27,2019-05-27,1,1,5.0,128.0,132.0,0,0,...,0,0,0,0,0,0,0,0,ITA,RM
1,2,2019-05-31,2019-06-13,14,3,2.0,156.0,144.0,0,0,...,0,0,0,0,0,0,0,0,ITA,RN
2,3,2019-04-26,2019-10-06,46,5,1.0,128.0,131.0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,4,2019-05-19,2019-05-19,1,1,1.0,173.0,133.0,0,0,...,1,0,0,0,0,0,0,0,NaN,NaN
4,5,2019-05-06,2019-10-06,6,2,2.0,255.0,20.0,0,0,...,0,0,0,0,0,0,0,0,ITA,PV
5,6,2019-06-06,2019-06-06,1,1,2.0,11.0,124.0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
6,7,2019-05-15,2019-05-20,6,2,2.0,11.0,143.0,0,0,...,0,0,0,0,0,0,0,0,ITA,FI
7,8,2019-06-15,2019-06-15,1,1,2.0,86.0,124.0,0,0,...,0,0,0,0,0,0,0,0,ITA,MI
8,9,2019-06-16,2019-06-16,1,1,2.0,11.0,124.0,0,0,...,0,1,0,0,0,0,0,0,ITA,MB


*DATA QUALITY CHECKS & DATA CLEANING*

In [6]:
# create a method that checks for each dataset which columns has null values
def null_value_check(df):
    # loop through each column in the DataFrame
    for col in df.columns:
        # check if there are any null values in the column
        if df[col].isnull().any():
            print(f"for dataset {df} Column {col} has null values.".format(df=df))
        else:
            pass

#check for NaN values through dataset
datasets = [dataset_csv,meta_os,meta_device,meta_browser]
for i in datasets:
    print( null_value_check(i))

#replace null with dummy value "*"
dataset_csv["device_type"].fillna("*",inplace = True)
dataset_csv["browser_type"].fillna("*",inplace = True)
dataset_csv["os_type"].fillna("*",inplace = True)
dataset_csv["prima_citta_stato_cod"].fillna("*",inplace = True)
dataset_csv["prima_citta_cod"].fillna("*",inplace = True)





#remove rows that contains null values from the final fact table, create an error table for further analysis

q = "select * FROM dataset_csv where device_type = '*' or browser_type = '*' or os_type = '*' or prima_citta_stato_cod = '*' or prima_citta_cod = '*'"

dataset_csv_errors = sqldf(q, globals())

dataset_csv = dataset_csv[~dataset_csv.user_id.isin(dataset_csv_errors["user_id"])]

#cast float columns as integer
dataset_csv['device_type'] = dataset_csv['device_type'].astype(int)
dataset_csv['browser_type'] = dataset_csv['browser_type'].astype(int)
dataset_csv['os_type'] = dataset_csv['os_type'].astype(int)

#PRIMARY KEY CHECKS

pk_checks=[]

# create a method that scan checks in order to find errors

def scan_checks(check_type, dataframes):
  fails=[]
  for i in dataframes:
    if i.empty:
      pass
    else:
      print("{check_type} failed for the following dataset: ".format(check_type=check_type),i.name)
      fails.append(i)
  if len(fails)>0:
    pass
  else:
    print("{check_type} checks ended with success!!".format(check_type=check_type))


#primary key checks on meta_browser
q = "SELECT id, count(*) FROM meta_browser group by id having count(*)>1"
meta_browser_pk_check = sqldf(q, globals())
pk_checks.append(meta_browser_pk_check)

#primary key checks on meta_device
q = "SELECT id, count(*) FROM meta_device group by id having count(*)>1"
meta_device_pk_check = sqldf(q, globals())
pk_checks.append(meta_device_pk_check)

#primary key checks on meta_os
q = "SELECT id, count(*) FROM meta_os group by id having count(*)>1"
meta_os_pk_check = sqldf(q, globals())
pk_checks.append(meta_os_pk_check)

#primary key checks on dataset_csv
q = "SELECT user_id, count(user_id) FROM dataset_csv group by user_id having count(*)>1"
dataset_csv_pk_check = sqldf(q, globals())
pk_checks.append(dataset_csv_pk_check)

#FOREIGN KEY CHECKS

fk_checks=[]

#foreign key checks on device_type
q = "select * FROM (SELECT * FROM dataset_csv as a left join meta_device as b  on a.device_type=b.id) where device_type is null"
meta_device_fk_check = sqldf(q, globals())
fk_checks.append(meta_device_fk_check)

#foreign key checks on browser_type
q = "select * FROM (SELECT * FROM dataset_csv as a left join meta_browser as b  on a.browser_type=b.id) where browser_type is null"
meta_browser_fk_check = sqldf(q, globals())
fk_checks.append(meta_browser_fk_check)

#foreign key checks on os_type
q = "select * FROM (SELECT * FROM dataset_csv as a left join meta_os as b  on a.os_type=b.id) where os_type is null"
meta_os_fk_check = sqldf(q, globals())
fk_checks.append(meta_os_fk_check)

#SCAN CHECKS
scan_checks("PK", pk_checks)
scan_checks("FK",fk_checks)



None
None
None
None


<ipython-input-6-b081b182cebd>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_csv['device_type'] = dataset_csv['device_type'].astype(int)
<ipython-input-6-b081b182cebd>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_csv['browser_type'] = dataset_csv['browser_type'].astype(int)
<ipython-input-6-b081b182cebd>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

PK checks ended with success!!
FK checks ended with success!!


In [139]:
#download final datasets

dataset_csv.head()
dataset_csv_errors.head()

name="dataset_csv_final.csv"
df = dataset_csv.to_csv(name)
files.download(name)

name_error="dataset_csv_errors.csv"
df = dataset_csv_errors.to_csv(name_error)
files.download(name_error)


# meta_browser_final = pd.read_csv(meta_browser)
# meta_device_final = pd.read_csv(meta_device)
# meta_os_final = pd.read_csv(meta_os)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>